In [1]:
import numpy as np
import ht
import matplotlib.pyplot as plt
import CoolProp as cp
from IPython.display import Latex

#### Práctica
	
A un haz de tubos horizontales está formado por un patrón en cuadrado de 5 tubos en altura y 5 tubos a lo
	ancho. Los tubos de acero al carbono (\(k_w\) =50W/m K) son de diámetro externo $D=$25mm, con un espesor de
	pared \(e_t =1\)mm. Estos se encuentran sumergidos en una corriente de vapor saturado a \(T_{sat} =350\)K. Por uno de
	sus extremos ingresa agua a \(T_{AE}=300\)K, para un caudal que en el interior de estos se establece una
	velocidad de \(V=2\)m/s (un paso por tubos). Si se requiere que la temperatura del agua a la salida sea
	\(T_{AS}=320\)K, se pide calcular:

1.  El caudal de agua establecido (\(\dot m_f\) ).
2.  El calor intercambiado (\(\dot Q\)).
3.  La masa de vapor condensado (\(\dot m_c\) )
4.  El coeficiente global de transmisión (\(U\)).
5.  La superficie de intercambio requerida (\(A\)).
	Durante el funcionamiento, repentinamente la temperatura del agua cae a \(T_{AE}'= 295\)K. Asumiendo que todos
	los parámetros calculados (\(U, A, \dot m_f\) , y todas las propiedades de los fluidos) permanecen constantes,
	determinar:
6.  La nueva temperatura del agua a la salida (\(T_{AS}'\))
7.  El nuevo calor intercambiado (\(\dot Q'\)).
8.  La nueva tasa de condensado (\(\dot m_c'\) ).

In [2]:
nf, nc = [5, 5]
ntubos = nf * nc
D = 25e-3
e_t = 1e-3
Tae = 300
V = 2
Tsat = 350
Tas = 320

In [3]:
rho_v = cp.CoolProp.PropsSI('D', 'T', Tsat, 'Q', 1, 'Water')
cp_v = cp.CoolProp.PropsSI('C', 'T', Tsat, 'Q', 1, 'Water')
h_v = cp.CoolProp.PropsSI('H', 'T', Tsat, 'Q', 1, 'Water')
h_a = cp.CoolProp.PropsSI('H', 'T', Tsat, 'Q', 0, 'Water')
h_fg = h_v - h_a

acero = ht.nearest_material('metals, steel')
k_w = ht.k_material(acero)

Tam = (Tas + Tae) / 2
rho_a = cp.CoolProp.PropsSI('D', 'T', Tam, 'Q', 0, 'Water')
cp_a = cp.CoolProp.PropsSI('C', 'T', Tam, 'Q', 0, 'Water')
mu_a = cp.CoolProp.PropsSI('V', 'T', Tam, 'Q', 0, 'Water')
k_a = cp.CoolProp.PropsSI('L', 'T', Tam, 'Q', 0, 'Water')
Pr_a = cp.CoolProp.PropsSI('Prandtl', 'T', Tam, 'Q', 0, 'Water')
nu_a = mu_a / rho_a
Area_a = np.pi * (D - 2 * e_t)**2 / 4 * ntubos
G_a = V * Area_a
m_a = G_a * rho_a
Q_a = m_a * cp_a * np.abs(Tae - Tas)

In [4]:
display(Latex(f'caudal másico de agua: $\dot m_a={m_a:.2f}$ kg/s'))
display(Latex(f'Calor Intercambiado: $Q_a={Q_a*1e-3:.2f}$ kW'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
m_c = Q_a/h_fg
Tw = (Tam + Tsat)/2
display(Latex(f'Masa condensado: $\dot m_c={m_c:.2f}$ kg/s'))

<IPython.core.display.Latex object>

In [6]:
rho_l = cp.CoolProp.PropsSI('D','T',Tsat,'Q',0,'Water')
k_l = cp.CoolProp.PropsSI('L','T',Tsat,'Q',0,'Water')
mu_l = cp.CoolProp.PropsSI('V','T',Tsat,'Q',0,'Water')
h_cond = ht.condensation.Nusselt_laminar(Tsat, Tw, rho_v, rho_l, k_l, mu_l, h_fg, nf*D)*.728/.943 #correcion tubo/placa

In [7]:
display(Latex(f'Coeficiente transferencia por condensación: $h_{{cond}}={h_cond:.2f}$ watt/m$^2$ K'))

<IPython.core.display.Latex object>

In [8]:
Re_a = V*(D-2*e_t)/nu_a
Nu_a = ht.conv_internal.Nu_conv_internal(Re_a, Pr_a)
h_a = Nu_a * k_a/(D-2*e_t)

In [9]:
display(Latex(f'Coeficiente transferencia por convección: $h_{{conv}}={h_a:.2f}$ watt/m$^2$ K'))

<IPython.core.display.Latex object>

In [10]:
U = 1 / ( (1/h_cond ) + (D*np.log(D/(D-2*e_t))/(2*k_w)) + (D/h_a/(D-2*e_t)) )

In [11]:
display(Latex(f'Coeficiente transferencia Global: $U={U:.2f}$ watt/m$^2$ K'))

<IPython.core.display.Latex object>

In [12]:
DTLM = ht.LMTD(Tsat, Tsat, Tae, Tas, counterflow=True)
display(Latex(f'DTLM ={DTLM:.1f} K'))

<IPython.core.display.Latex object>

In [13]:
A_total = Q_a / (U*DTLM)
display(Latex(f'Area total ={A_total:.1f} m$^2$'))

<IPython.core.display.Latex object>

In [14]:
Cmin = cp_a*m_a
Nut = U*A_total / Cmin
display(Latex(f'Nut ={Nut:.2f}'))
epsilon = ht.effectiveness_from_NTU(Nut,0)
display(Latex(f'\(\\varepsilon =\){epsilon:.2f}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [15]:
Tae2 = 295
Tas2 = Tae2+(Tsat-Tae2)*epsilon
display(Latex(f'\(T_{{as}}^\prime =\){Tas2:.1f}K'))
Q_a2 = m_a * cp_a*(Tas2-Tae2)
display(Latex(f'\(\dot Q^\prime =\){Q_a2*1e-3:.1f}kW'))
m_c2 = Q_a2/h_fg
display(Latex(f'Masa condensado nueva: $\dot m_c^\prime={m_c2:.2f}$ kg/s'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>